# Update NHL Data

This Jupyter Notebook is scheduled to run every day, scraping MoneyPuck, Odds, and historical game data from MoneyPuck, ESPN, and Hockey Reference. The results are stored in csv files to be manually committed to the github when convienient.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import datetime as dt

#!pip install selenium
#!pip install webdriver_manager
#!pip install jupyter_scheduler
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 

## Scrape MoneyPuck Data

In [2]:
def get_todays_matchups():
    date = dt.date.today()
    url = "https://moneypuck.com/index.html?date=" + str(date)
    return url

def update_matchups(matchup_df, url=get_todays_matchups()):
    options = webdriver.ChromeOptions()  # instantiate options 
    options.headless = True  # run browser in headless mode 
    # instantiate driver 
    driver = webdriver.Chrome() 

    # load website  
    driver.get(url) # get the entire website content 

    # select win probability table
    win_table = driver.find_elements(By.ID, 'includedContent')[0] 
    games = win_table.find_elements(By.TAG_NAME, "tr")
    date = str(dt.date.today())
    for game in games:
        win_probs = game.find_elements(By.TAG_NAME, "h2")
        teams = game.find_elements(By.TAG_NAME, "img")
        away_win_prob = win_probs[0].text
        home_win_prob = win_probs[1].text
        away_team = teams[0].get_attribute("alt")
        home_team = teams[1].get_attribute("alt")
        away_win_prob = float(away_win_prob.replace("%",""))/100 #Remove % symbol and convert to float (0-1)
        home_win_prob = float(home_win_prob.replace("%",""))/100
        row = {"date":date, "away_team":away_team, "away_win_prob":away_win_prob, "home_team":home_team, "home_win_prob":home_win_prob}
        matchup_df = matchup_df.append(row, ignore_index=True)
    
    driver.close()
    return matchup_df

In [3]:
moneypuck_df = pd.read_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Historical_Moneypuck_Predictions.csv", index_col=0)
try:
    moneypuck_df = update_matchups(moneypuck_df)
except:
    print("Web-scraping error with Moneypuck site")
moneypuck_df.tail(10)

,date,away_team,away_win_prob,home_team,home_win_prob
64,2023-11-14,VEGAS GOLDEN KNIGHTS,0.570,WASHINGTON CAPITALS,0.430
65,2023-11-14,ARIZONA COYOTES,0.349,DALLAS STARS,0.651
66,2023-11-14,ANAHEIM DUCKS,0.352,NASHVILLE PREDATORS,0.648
67,2023-11-14,TAMPA BAY LIGHTNING,0.558,ST. LOUIS BLUES,0.442
68,2023-11-14,NEW JERSEY DEVILS,0.442,WINNIPEG JETS,0.558
69,2023-11-14,FLORIDA PANTHERS,0.722,SAN JOSE SHARKS,0.278
70,2023-11-15,PHILADELPHIA FLYERS,0.330,CAROLINA HURRICANES,0.670
71,2023-11-15,SEATTLE KRAKEN,0.332,EDMONTON OILERS,0.668
72,2023-11-15,ANAHEIM DUCKS,0.270,COLORADO AVALANCHE,0.730
73,2023-11-15,NEW YORK ISLANDERS,0.404,VANCOUVER CANUCKS,0.596


In [4]:
moneypuck_df.to_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Historical_Moneypuck_Predictions.csv")

### Scrape Daily Lines

In [5]:
def convert_odds(odds):
    '''Converts a pandas series of odds from american odds to decimal odds'''
    if odds < 0: #If team is the favorite (negative american odds)
        decimal_odds = -1/(odds/100) + 1
    else:
        decimal_odds = (odds/100) + 1
    return decimal_odds

def update_odds(odds_df):
    options = webdriver.ChromeOptions()  # instantiate options 
    options.headless = True  # run browser in headless mode 
    # instantiate driver 
    driver = webdriver.Chrome(options=options)
    driver.get('https://www.espn.com/nhl/lines') # get the entire website content

    date = str(dt.date.today())
    games = driver.find_elements(By.TAG_NAME, 'tr')
    i = 0
    for game in games:
        game_str = game.text
        if i % 3 != 0: #Skip the header of every table
            team_data = game_str.splitlines()
            odds_line = team_data[-1] #Line containing each team's odds is the last of the table
            odds_line = odds_line.split(" ")
            if len(odds_line) == 6: #The over is always listed on the away team's row
                away_team = team_data[0].upper()
                away_odds = convert_odds(int(odds_line[2])) #Have to account for the goalie name being split
                over_under = float(odds_line[3])
                away_puck_line = float(odds_line[4])
                away_puck_odds = convert_odds(int(odds_line[5]))
            else:
                home_team = team_data[0].upper()
                home_odds = convert_odds(int(odds_line[2]))
                home_puck_line = float(odds_line[3])
                home_puck_odds = convert_odds(int(odds_line[4]))
        elif i > 0:
            row = {"date": date, "away_team":away_team, "away_odds":away_odds, "home_team":home_team, "home_odds":home_odds,
                   "over_under":over_under, "away_puck_line":away_puck_line, "away_puck_odds":away_puck_odds, 
                   "home_puck_line":home_puck_line, "home_puck_odds":home_puck_odds}
            odds_df = odds_df.append(row, ignore_index=True)
        i += 1
    driver.close()
    return odds_df

In [7]:
odds_df = pd.read_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Historical_Odds.csv", index_col=0)
try:
    odds_df = update_odds(odds_df)
except:
    print("Web-scraping error with ESPN Odds")
odds_df.tail(10)

IndexError: list index out of range

In [36]:
odds_df.to_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Historical_Odds.csv")

### Scrape Past Games

In [8]:
def update_past_games(historical_df):
    hockey_scores = requests.get("https://www.hockey-reference.com/boxscores/")
    soup = BeautifulSoup(hockey_scores.text)
    games = soup.find_all(attrs={"class":"teams"})
    date = str(dt.date.today() - dt.timedelta(days=1))
    for game in games: 
        teams = game.find_all("a")[::2] #Ignore the "Final", since it's not a team
        scores = game.find_all(attrs={"class":"right"})[:3:2]
        i = 0
        for team, score in zip(teams,scores):
            if i % 2 == 0:
                away_team = team.get_text().upper()
                away_score = score.get_text()
            else:
                home_team = team.get_text().upper()
                home_score = score.get_text()
                row = {"date": date, "away_team":away_team, "away_score":away_score, "home_team":home_team, "home_score":home_score}
                historical_df = historical_df.append(row, ignore_index=True)
            i += 1

    return historical_df

In [9]:
historical_df = pd.read_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Game_Outcomes.csv", index_col=0)
try:
    historical_df = update_past_games(historical_df)
except:
    print("Web-scraping error with Hockey Reference")
historical_df.tail()

,date,away_team,away_score,home_team,home_score
82,2023-11-14,ANAHEIM DUCKS,3,NASHVILLE PREDATORS,2
83,2023-11-14,FLORIDA PANTHERS,5,SAN JOSE SHARKS,3
84,2023-11-14,TAMPA BAY LIGHTNING,0,ST. LOUIS BLUES,5
85,2023-11-14,NEW JERSEY DEVILS,3,WINNIPEG JETS,6
86,2023-11-14,VEGAS GOLDEN KNIGHTS,0,WASHINGTON CAPITALS,3


In [10]:
historical_df.to_csv("C:\\Users\\geniu\\Documents\\NHL-Predictions\\NHL-Predictions\\Game_Outcomes.csv")